In [ ]:
#imports
import copy #for deepcopy
import threading

#self-written shortcuts
from Class_Core    import *
from Class_FI      import *
from Class_IBKR    import *
from Class_xlWings import *

core = Core_Methods()
xlw  = xlWings()


In [ ]:
class Local_IBKR(IBKR):
    
    def __init__(self):
        super().__init__()
        self.dict_of_details = {}
        
    def contractDetails(self, reqId, contractDetails):
        super().contractDetails(reqId, contractDetails)
        self.dict_of_details[contractDetails.contract.conId] = vars(contractDetails)

    #  below should be in local versions since they depend on local variable names    
    def tickPrice(self, reqId, field, price, attribs):
        super().tickPrice(reqId, field, price, attribs)
        if field in [1, 2, 9]:
            obj = self.fi_objs_dict[reqId]
            obj.prices.make_price_updates(field, price)    
        
    def tickSize(self, reqId, field, size):
        super().tickSize(reqId, field, size)
        if field in [0, 3, 8, 21]:
            obj = self.fi_objs_dict[reqId]
            obj.sizes.make_size_updates(field, size)    
            
ibkr = Local_IBKR()


In [ ]:
def get_fin_inst_data(data_type, config, simple_df=None):
    df            = make_instruments_df(data_type, config)
    df            = prep_instruments_df(data_type, df)
    df            = make_contracts(data_type, df, simple_df)
    if data_type == 'simple':
        df = req_simple_contract_details(df, config['timer_interval'])
    return df
    
    
def make_instruments_df(data_type, config):
    if data_type   == 'simple':
        ibkr.connect_to_ibkr(config['ibkr_port'], config['ibkr_channel'])
        dfs         = []
        for label in ['futs', 'etfs']:
            df      = xlw.getDF(config['workbook_name'], config[f'{label}_input_worksheet'], config[f'{label}_input_range'])
            dfs.append(df)
        df          = pd.concat(dfs, ignore_index=True)
        
    elif data_type == 'complex':
        df          = xlw.getDF(config['workbook_name'], config['spds_input_worksheet'], config['spds_input_range'])
    
    print(f'{data_type.capitalize()} instrument data retrieved from spreadsheet', '\n')
    return df    

    
def prep_instruments_df(data_type, df):
    df                    = df[df['get_price']].copy()
    df.drop(columns=['get_price'], inplace=True)
    df.reset_index(drop=True, inplace=True)  
    df['reqId']           = df.index.astype("Int64")
    df['s_or_c']          = data_type
    
    if data_type == 'simple':
        df['ibkr_conId']  = df['ibkr_conId'].astype("Int64")
    
    elif data_type == 'complex':
        df['ibkr_conId1'] = df['ibkr_conId1'].astype("Int64")
        df['ibkr_conId2'] = df['ibkr_conId2'].astype("Int64")   
    
    print(f'{data_type.capitalize()} instrument dataframe created', '\n')    
    return df


def make_contracts(data_type, df, simple_df=None):
    df                     = df.copy()
    
    if data_type          == 'simple':
        df['contract_obj'] = df.apply(lambda row: ibkr.makeSimpleContract(row.ibkr_conId), axis=1)
    
    elif data_type        == 'complex':
        contracts_list     = []
        for row in df.itertuples(index=False):  
            c1             = simple_df.loc[simple_df["ibkr_conId"] == row.ibkr_conId1, 'contract_dict'].iloc[0]
            c2             = simple_df.loc[simple_df["ibkr_conId"] == row.ibkr_conId2, 'contract_dict'].iloc[0]   
            sym            = c1['symbol']
            fi_list_1      = [row.ibkr_conId1, c1['exchange'], row.ratio1, row.action1]   
            fi_list_2      = [row.ibkr_conId2, c2['exchange'], row.ratio2, row.action2]            
            contract       = ibkr.makeComplexContract(sym, fi_list_1, fi_list_2)   
            contracts_list.append(contract)
        df['contract_obj'] = contracts_list
    
    print(f'{data_type.capitalize()} instrument contracts created', '\n')        
    return df
   

def req_simple_contract_details(df, timer_interval):
    threading.Timer(2, ibkr.run).start()
    print('IBKR running', '\n')
    
    for row in df.itertuples(index=True):
        ibkr.reqContractDetails(row.reqId, row.contract_obj)
    print('Simple instrument contract details requested', '\n')
    
    time.sleep(timer_interval)
    
    ibkr.disconnect()
    print('IBKR disconnected', '\n')

    df = df.copy()
    df['details_dict']  = df['ibkr_conId'].map(ibkr.dict_of_details)
    df['contract_obj']  = df['details_dict'].apply(lambda d: d['contract'] if d else None)
    df['contract_dict'] = df['contract_obj'].apply(lambda d: vars(d) if d else None) 
    
    print('Simple instrument contract details received', '\n')     
    return df


In [ ]:
def print_mkt_data_timer(config, objs_dict):
    while True: 
        time.sleep(config['timer_interval'])
        print("\n", datetime.now()) 
        print_mkt_data(config, objs_dict)

def print_mkt_data(config, obj_dict):
    df = xlw.dictToDF(obj_dict.copy())
    df = df.sort_values(['reqId'])

    cols_flatten = ['details_dict', 'prices', 
                    'prices.mkt_price_dict', 'prices.total_cf_unit_dict', 'prices.comm_unit_dict'] #'contract_dict', 'my_multipliers', 'sizes']:
    for col in cols_flatten:
        if col in df.columns:
            df = xlw.flattenColumn(df, col, sep='.') 

    cols_order  = ['prices.mkt_price_dict.bid', 	 
                   'prices.total_cf_unit_dict.hit_bid',
                   'my_symbol',
                   'details_dict.realExpirationDate',
                   'prices.total_cf_unit_dict.lift_ask',
                   'prices.mkt_price_dict.ask',
                   'prices.comm_unit_dict.join_bid'
                  ]

    xlw.printDFToXL(config['workbook_name'], 
                    config['prices_worksheet'], 
                    config['prices_range'], 
                    df[cols_order])


In [ ]:
def get_started():
    print("\n")    
    answer = '2026 Spreads - Futures and ETFs' # input("Which Excel Workbook? ")    
    worksheet_name         = 'Essentials'
    worksheet_cell         = 'B1'   
    workbook_name = answer + ".xlsm"
    print("\n") 
    xlRange                = xlw.getCell(workbook_name, worksheet_name, worksheet_cell)
    dict_                  = xlw.getDict(workbook_name, worksheet_name, xlRange)
    dict_['ibkr_channel']  = ibkr.get_ibkr_channel()
    print('Initialization complete','\n')
    return dict_


def get_fi_data(config):            
    simple_df  = get_fin_inst_data('simple', config)  
    complex_df = get_fin_inst_data('complex', config, simple_df)      
    df = pd.concat([simple_df, complex_df], ignore_index=True, copy=False)
    df['reqId'] = df.index.astype("Int64")
    print('Financial instrument data gathered','\n')
    return df


def make_fi_objs_simple(df):
    objs_dict = {}    
    df_simple = df[df['s_or_c'] == 'simple']
    for row in df_simple.itertuples(index=False):
        info = row._asdict()        
        obj = Financial_Instrument(info)
        objs_dict[row.reqId] = obj
    print('Simple financial instrument objects created','\n')    
    return objs_dict


def make_fi_objs_complex(df, simple_dict):
    temp_dict = {} 
    for obj in simple_dict.values(): 
        temp_dict[obj.contract_obj.conId] = obj        
    objs_dict = {}
    df_complex = df[df['s_or_c'] == 'complex']    
    for row in df_complex.itertuples(index=False):
        info = row._asdict()        
        obj1 = temp_dict.get(row.ibkr_conId1)
        obj2 = temp_dict.get(row.ibkr_conId2)
        obj  = Financial_Instrument(info, obj1, obj2)
        objs_dict[row.reqId] = obj       
    print('Complex financial instrument objects created','\n')    
    return objs_dict
    

def print_details(config, dict_):
    df = xlw.dictToDF(dict_)

    cols_flatten = ['details_dict', 'contract_dict', 'my_multipliers', 'dates', 'prices', 'sizes']
    for col in cols_flatten:
        if col in df.columns:
            df = xlw.flattenColumn(df, col, sep='.') 
    df = df.drop(columns=['contract_obj', 'details_dict.contract', 'dates.fi'], errors='ignore')
  
    for col in ['details_dict.secIdList', 'my_comm_list', 'prices.comm_list']:
        df[col] = df[col].astype("string")    
        if col == 'details_dict.secIdList':
            df[col] = ('[' + df[col].str.split(' ', n=1).str[-1]).where(~df[col].isin(['<NA>', 'nan', 'None']), None)
            
#    print(df.columns.tolist())
#    xlw.find_print_problems(df)
    
    threading.Thread(target = xlw.printDFToXL, 
                     args = (config['workbook_name'], 
                             config['details_output_worksheet'], 
                             config['details_output_range'], 
                             df), daemon=True).start()   
   
    print('Financial instrument details printed', '\n')


def show_prices(config, objs_dict):
    threading.Thread(target = print_mkt_data_timer, args = (config, objs_dict)).start()
    ibkr.connect_to_ibkr(config['ibkr_port'], config['ibkr_channel'])
    time.sleep(2)
    
    for obj in objs_dict.values():
        ibkr.reqMktData(obj.reqId, obj.contract_obj, "", False, False, [])   
    print('Starting to show prices','\n') 
    ibkr.run()   


In [ ]:
def main():    
    config            = get_started() 
    df                = get_fi_data(config) 
    simple_objs_dict  = make_fi_objs_simple(df)
    complex_objs_dict = make_fi_objs_complex(df, simple_objs_dict)
    ibkr.fi_objs_dict = simple_objs_dict | complex_objs_dict
    print_details(config, ibkr.fi_objs_dict)
    show_prices(config, ibkr.fi_objs_dict)

main()
